# Load Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

from google.cloud import bigquery
from google.oauth2 import service_account

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive/')

import scipy.stats as stats
from scipy.stats import chi2_contingency
from statsmodels.stats import proportion

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Connect to BigQuery

In [2]:
FOLDERPATH = '/content/drive/.../config'

KEY_NAME = 'key.json'
KEY_PATH = FOLDERPATH + '/' + KEY_NAME

credentials = service_account.Credentials.from_service_account_file(KEY_PATH)

client = bigquery.Client(
    credentials = credentials,
    project = credentials.project_id
)

TABLE = 'project_id.dataset_id.events_*'

# Experiment Info

In [3]:
EXPERIMENT_ID = 'experiment_id'
START_DATE, END_DATE = 'yyyy-mm-dd', 'yyyy-mm-dd'
CONFIDENCE_LEVEL = 0.95
ALPHA = 1 - CONFIDENCE_LEVEL

# Independence Test

### (1) Extract Raw Data

In [4]:
SQL = f"""
      WITH
      CTE_raw AS (
          SELECT
              DISTINCT
              user_pseudo_id,
              (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'variant_id') AS variant_id,
              geo.country,
              device.category AS device_type
          FROM
              `{TABLE}`
          WHERE
              _table_suffix BETWEEN FORMAT_DATE('%Y%m%d', '{START_DATE}') AND FORMAT_DATE('%Y%m%d', '{END_DATE}')
              AND event_name = 'experiment_impression'
              AND (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'experiment_id') = '{EXPERIMENT_ID}'
              AND user_pseudo_ID IS NOT NULL
      )
      SELECT
          user_pseudo_id,
          CASE
              WHEN ENDS_WITH(variant_id, '.0') THEN 'A'
              WHEN ENDS_WITH(variant_id, '.1') THEN 'B'
          END AS test_group,
          CASE
              WHEN country IN ('South Korea', 'United States') THEN country
              ELSE 'Others'
          END AS country,
          CASE
              WHEN device_type IN ('desktop', 'mobile', 'tablet') THEN device_type
              ELSE 'others'
          END AS device_type
      FROM
          CTE_raw
"""

results = client.query(SQL)
independence_df = results.to_dataframe()

### (2) Independence Test

* Country

In [5]:
countries = independence_df[['test_group', 'country']]
contingency_table = pd.crosstab(
    countries['test_group'],
    countries['country']
)

chi_value, p_value, df, expected = chi2_contingency(contingency_table)
print('chi-squared:', np.round(chi_value, 4))
print('p-value:', np.round(p_value, 4))

if p_value < ALPHA:
  print('Countries and Test Groups are Related.')
else:
  print('Countries and Test Groups are Independent.')

chi-squared: 1.7531
p-value: 0.4162
Countries and Test Groups are Independent.


* Device Type

In [6]:
device_types = independence_df[['test_group', 'device_type']]
contingency_table = pd.crosstab(
    device_types['test_group'],
    device_types['device_type']
)

chi_value, p_value, df, expected = chi2_contingency(contingency_table)
print('chi-squared:', np.round(chi_value, 4))
print('p-value:', np.round(p_value, 4))

if p_value < ALPHA:
  print('Countries and Test Groups are Related.')
else:
  print('Countries and Test Groups are Independent.')

chi-squared: 4.9183
p-value: 0.0855
Countries and Test Groups are Independent.


# Extract Raw Data

### (1) Purchase CVR, Add-to-cart CVR, Begin-checkout CVR

In [7]:
EVENT_NAMES = ['purchase', 'add_to_cart', 'begin_checkout']

for EVENT_NAME in EVENT_NAMES:
  SQL = f"""
        WITH
        CTE_raw AS (
            SELECT
                user_pseudo_id,
                event_timestamp,
                event_name,
                (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'experiment_id') AS experiment_id,
                (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'variant_id') AS variant_id
            FROM
                `{TABLE}`
            WHERE
                _table_suffix BETWEEN FORMAT_DATE('%Y%m%d', '{START_DATE}') AND FORMAT_DATE('%Y%m%d', '{END_DATE}')
                AND event_name IN ('experiment_impression', '{EVENT_NAME}')
                AND user_pseudo_ID IS NOT NULL
                AND event_timestamp IS NOT NULL
        ),
        CTE_users_assigned AS (
            SELECT
                user_pseudo_id,
                CASE
                    WHEN ENDS_WITH(variant_id, '.0') THEN 'A'
                    WHEN ENDS_WITH(variant_id, '.1') THEN 'B'
                END AS test_group,
                MIN(event_timestamp) AS assigned_timestamp
            FROM
                CTE_raw
            WHERE
                event_name = 'experiment_impression'
                AND experiment_id = '{EXPERIMENT_ID}'
                AND variant_id IS NOT NULL
            GROUP BY
                1, 2
        ),
        CTE_users_converted AS (
            SELECT
                user_pseudo_id,
                event_timestamp AS converted_timestamp
            FROM
                CTE_raw
            WHERE
                event_name = '{EVENT_NAME}'
        ),
        CTE_users_assigned_converted AS (
            SELECT
                DISTINCT
                A.user_pseudo_id
            FROM
                CTE_users_converted C
            LEFT JOIN
                CTE_users_assigned A
                ON C.user_pseudo_id = A.user_pseudo_id
                AND C.converted_timestamp >= A.assigned_timestamp
        )
        SELECT
            A.user_pseudo_id,
            A.test_group,
            CASE
                WHEN AC.user_pseudo_id IS NOT NULL THEN 1
                ELSE 0
            END AS converted
        FROM
            CTE_users_assigned A
        LEFT JOIN
            CTE_users_assigned_converted AC
            ON A.user_pseudo_id = AC.user_pseudo_id
  """

  results = client.query(SQL)
  if EVENT_NAME == 'purchase':
    purchase_cvr_df = results.to_dataframe()
  elif EVENT_NAME == 'add_to_cart':
    addtocart_cvr_df = results.to_dataframe()
  elif EVENT_NAME == 'begin_checkout':
    begincheckout_cvr_df = results.to_dataframe()

### (2) Other Platforms CVR

In [8]:
SQL = f"""
      WITH
      CTE_raw AS (
          SELECT
              user_pseudo_id,
              event_timestamp,
              event_name,
              REPLACE(
                  REGEXP_REPLACE(
                      REGEXP_REPLACE(
                          LOWER((SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'page_location')),
                          r'(\?.*)$', ''
                      ),
                      r'/$', ''
                  ),
                  'https://', ''
              ) AS page_location,
              REPLACE(
                  REGEXP_REPLACE(
                      REGEXP_REPLACE(
                          LOWER((SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'link_url')),
                          r'(\?.*)$', ''
                      ),
                      r'/$', ''
                  ),
                  'https://', ''
              ) AS link_url,
              (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'experiment_id') AS experiment_id,
              (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'variant_id') AS variant_id
          FROM
              `{TABLE}`
          WHERE
              _table_suffix BETWEEN FORMAT_DATE('%Y%m%d', '{START_DATE}') AND FORMAT_DATE('%Y%m%d', '{END_DATE}')
              AND event_name IN ('experiment_impression', 'click')
              AND user_pseudo_ID IS NOT NULL
              AND event_timestamp IS NOT NULL
      ),
      CTE_users_assigned AS (
          SELECT
              user_pseudo_id,
              CASE
                  WHEN ENDS_WITH(variant_id, '.0') THEN 'A'
                  WHEN ENDS_WITH(variant_id, '.1') THEN 'B'
              END AS test_group,
              MIN(event_timestamp) AS assigned_timestamp
          FROM
              CTE_raw
          WHERE
              event_name = 'experiment_impression'
              AND experiment_id = '{EXPERIMENT_ID}'
              AND variant_id IS NOT NULL
          GROUP BY
              1, 2
      ),
      CTE_users_converted AS (
          SELECT
              user_pseudo_id,
              event_timestamp AS converted_timestamp
          FROM
              CTE_raw
          WHERE
              event_name = 'click'
              AND page_location LIKE '%/shoplist%'
              AND (
                link_url LIKE '%amazon.com%'
                OR link_url LIKE '%smartstore.naver.com%'
                OR link_url LIKE '%coupang.com%'
                OR link_url LIKE '%auction.co.kr%'
                OR link_url LIKE '%11st.co.kr%'
                OR link_url LIKE '%gmarket.co.kr%'
              )
      ),
      CTE_users_assigned_converted AS (
          SELECT
              DISTINCT
              A.user_pseudo_id
          FROM
              CTE_users_converted C
          LEFT JOIN
              CTE_users_assigned A
              ON C.user_pseudo_id = A.user_pseudo_id
              AND C.converted_timestamp >= A.assigned_timestamp
      )
      SELECT
          A.user_pseudo_id,
          A.test_group,
          CASE
              WHEN AC.user_pseudo_id IS NOT NULL THEN 1
              ELSE 0
          END AS converted
      FROM
          CTE_users_assigned A
      LEFT JOIN
          CTE_users_assigned_converted AC
          ON A.user_pseudo_id = AC.user_pseudo_id
"""

results = client.query(SQL)
otherplatforms_cvr_df = results.to_dataframe()

### (3) ARPU

In [9]:
SQL = f"""
      WITH
      CTE_raw AS (
          SELECT
              user_pseudo_id,
              event_timestamp,
              event_name,
              ecommerce.purchase_revenue_in_usd AS revenue,
              (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'experiment_id') AS experiment_id,
              (SELECT value.string_value FROM UNNEST (event_params) WHERE key = 'variant_id') AS variant_id
          FROM
              `{TABLE}`
          WHERE
              _table_suffix BETWEEN FORMAT_DATE('%Y%m%d', '{START_DATE}') AND FORMAT_DATE('%Y%m%d', '{END_DATE}')
              AND event_name IN ('experiment_impression', 'purchase')
              AND user_pseudo_ID IS NOT NULL
              AND event_timestamp IS NOT NULL
      ),
      CTE_users_assigned AS (
          SELECT
              user_pseudo_id,
              CASE
                  WHEN ENDS_WITH(variant_id, '.0') THEN 'A'
                  WHEN ENDS_WITH(variant_id, '.1') THEN 'B'
              END AS test_group,
              MIN(event_timestamp) AS assigned_timestamp
          FROM
              CTE_raw
          WHERE
              event_name = 'experiment_impression'
              AND experiment_id = '{EXPERIMENT_ID}'
              AND variant_id IS NOT NULL
          GROUP BY
              1, 2
      ),
      CTE_users_revenue AS (
          SELECT
              user_pseudo_id,
              event_timestamp AS purchased_timestamp,
              revenue
          FROM
              CTE_raw
          WHERE
              event_name = 'purchase'
      ),
      CTE_users_assigned_revenue AS (
          SELECT
              A.user_pseudo_id,
              SUM(R.revenue) AS revenue
          FROM
              CTE_users_revenue R
          LEFT JOIN
              CTE_users_assigned A
              ON R.user_pseudo_id = A.user_pseudo_id
              AND R.purchased_timestamp > A.assigned_timestamp
          GROUP BY
              1
      )
      SELECT
          A.user_pseudo_id,
          A.test_group,
          COALESCE(AR.revenue, 0) AS revenue
      FROM
          CTE_users_assigned A
      LEFT JOIN
          CTE_users_assigned_revenue AR
          ON A.user_pseudo_id = AR.user_pseudo_id
"""

results = client.query(SQL)
arpu_df = results.to_dataframe()

# Analysis

### Key Metrics (1) Purchase CVR

* Assumption

In [10]:
na = purchase_cvr_df.groupby('test_group')['user_pseudo_id'].count()['A']
nb = purchase_cvr_df.groupby('test_group')['user_pseudo_id'].count()['B']

xa = purchase_cvr_df.groupby('test_group')['converted'].sum()['A']
xb = purchase_cvr_df.groupby('test_group')['converted'].sum()['B']

pa = xa / na
pb = xb / nb

if (na * pa < 5) or (na * (1 - pa) < 5) or (nb * pb < 5) or (nb * (1 - pb) < 5):
    print('Assumption Not Satisfied!')
else:
    print('Good to go!')

Good to go!


* Two-proportion z-test

In [11]:
z_result = proportion.proportions_ztest(
    count = [xa, xb],
    nobs = [na, nb],
    alternative = 'smaller'
)
z, p_value = z_result

print(f'Analysis Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Null hypothesis can be rejected. (A < B)')
else:
  print('- Null hypothesis can NOT be rejected. (A >= B)')

print('===============================================')

print(f'p-value = {p_value:.4f}')
print('- Purchase CVR (A):', f'{pa * 100:.2f}%')
print('- Purchase CVR (B):', f'{pb * 100:.2f}%')

Analysis Results (Confidence Level: 95%)
- Null hypothesis can NOT be rejected. (A >= B)
p-value = 0.0999
- Purchase CVR (A): 5.13%
- Purchase CVR (B): 6.32%


### Key Metrics (2) ARPU

* Assumption

In [12]:
na = arpu_df.groupby('test_group')['user_pseudo_id'].count()['A']
nb = arpu_df.groupby('test_group')['user_pseudo_id'].count()['B']

if na + nb < 30:
    print('Assumption Not Satisfied!')
else:
    print('Good to go!')

Good to go!


* Equal-variance Test: Bartlett's Test

In [13]:
stat, p_value = stats.bartlett(
    arpu_df[arpu_df['test_group'] == 'A']['revenue'].reset_index(drop=True),
    arpu_df[arpu_df['test_group'] == 'B']['revenue'].reset_index(drop=True)
)

print(f'Equal-variance Test Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Variances are NOT equal!')
  EQUAL_VAR = False
else:
  print('- Variances are equal!')
  EQUAL_VAR = True

Equal-variance Test Results (Confidence Level: 95%)
- Variances are equal!


* Two-sample t-test

In [14]:
t_result = stats.ttest_ind(
    arpu_df[arpu_df['test_group'] == 'A']['revenue'].reset_index(drop=True),
    arpu_df[arpu_df['test_group'] == 'B']['revenue'].reset_index(drop=True),
    equal_var = EQUAL_VAR,
    alternative = 'less'
)
t, p_value = t_result

print(f'Analysis Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Null hypothesis can be rejected. (A < B)')
else:
  print('- Null hypothesis can NOT be rejected. (A >= B)')

print('===============================================')

mean_a = arpu_df.groupby('test_group')['revenue'].mean()[0]
mean_b = arpu_df.groupby('test_group')['revenue'].mean()[1]

print(f'p-value = {p_value:.4f}')
print('- ARPU (A):', f'{mean_a:.2f}%')
print('- ARPU (B):', f'{mean_b:.2f}%')

Analysis Results (Confidence Level: 95%)
- Null hypothesis can NOT be rejected. (A >= B)
p-value = 0.2309
- ARPU (A): 4.69%
- ARPU (B): 5.43%


### Supplementary Metrics (1) Add-to-cart CVR

* Assumption

In [15]:
na = addtocart_cvr_df.groupby('test_group')['user_pseudo_id'].count()['A']
nb = addtocart_cvr_df.groupby('test_group')['user_pseudo_id'].count()['B']

xa = addtocart_cvr_df.groupby('test_group')['converted'].sum()['A']
xb = addtocart_cvr_df.groupby('test_group')['converted'].sum()['B']

pa = xa / na
pb = xb / nb

if (na * pa < 5) or (na * (1 - pa) < 5) or (nb * pb < 5) or (nb * (1 - pb) < 5):
    print('Assumption Not Satisfied!')
else:
    print('Good to go!')

Good to go!


* Two-proportion z-test

In [16]:
z_result = proportion.proportions_ztest(
    count = [xa, xb],
    nobs = [na, nb],
    alternative = 'smaller'
)
z, p_value = z_result

print(f'Analysis Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Null hypothesis can be rejected. (A < B)')
else:
  print('- Null hypothesis can NOT be rejected. (A >= B)')

print('===============================================')

print(f'p-value = {p_value:.4f}')
print('- Purchase CVR (A):', f'{pa * 100:.2f}%')
print('- Purchase CVR (B):', f'{pb * 100:.2f}%')

Analysis Results (Confidence Level: 95%)
- Null hypothesis can be rejected. (A < B)
p-value = 0.0438
- Purchase CVR (A): 1.52%
- Purchase CVR (B): 2.48%


### Supplementary Metrics (2) Begin-checkout CVR

* Assumption

In [17]:
na = begincheckout_cvr_df.groupby('test_group')['user_pseudo_id'].count()['A']
nb = begincheckout_cvr_df.groupby('test_group')['user_pseudo_id'].count()['B']

xa = begincheckout_cvr_df.groupby('test_group')['converted'].sum()['A']
xb = begincheckout_cvr_df.groupby('test_group')['converted'].sum()['B']

pa = xa / na
pb = xb / nb

if (na * pa < 5) or (na * (1 - pa) < 5) or (nb * pb < 5) or (nb * (1 - pb) < 5):
    print('Assumption Not Satisfied!')
else:
    print('Good to go!')

Good to go!


* Two-proportion z-test

In [18]:
z_result = proportion.proportions_ztest(
    count = [xa, xb],
    nobs = [na, nb],
    alternative = 'smaller'
)
z, p_value = z_result

print(f'Analysis Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Null hypothesis can be rejected. (A < B)')
else:
  print('- Null hypothesis can NOT be rejected. (A >= B)')

print('===============================================')

print(f'p-value = {p_value:.4f}')
print('- Purchase CVR (A):', f'{pa * 100:.2f}%')
print('- Purchase CVR (B):', f'{pb * 100:.2f}%')

Analysis Results (Confidence Level: 95%)
- Null hypothesis can NOT be rejected. (A >= B)
p-value = 0.1769
- Purchase CVR (A): 6.57%
- Purchase CVR (B): 7.52%


### Guardrail Metrics (1) Other Platforms CVR

* Assumption

In [19]:
na = otherplatforms_cvr_df.groupby('test_group')['user_pseudo_id'].count()['A']
nb = otherplatforms_cvr_df.groupby('test_group')['user_pseudo_id'].count()['B']

xa = otherplatforms_cvr_df.groupby('test_group')['converted'].sum()['A']
xb = otherplatforms_cvr_df.groupby('test_group')['converted'].sum()['B']

pa = xa / na
pb = xb / nb

if (na * pa < 5) or (na * (1 - pa) < 5) or (nb * pb < 5) or (nb * (1 - pb) < 5):
    print('Assumption Not Satisfied!')
else:
    print('Good to go!')

Good to go!


* Two-proportion z-test

In [20]:
z_result = proportion.proportions_ztest(
    count = [xa, xb],
    nobs = [na, nb],
    alternative = 'smaller'
)
z, p_value = z_result

print(f'Analysis Results (Confidence Level: {CONFIDENCE_LEVEL * 100:.0f}%)')

if p_value < ALPHA:
  print('- Null hypothesis can be rejected. (A < B)')
else:
  print('- Null hypothesis can NOT be rejected. (A >= B)')

print('===============================================')

print(f'p-value = {p_value:.4f}')
print('- Purchase CVR (A):', f'{pa * 100:.2f}%')
print('- Purchase CVR (B):', f'{pb * 100:.2f}%')

Analysis Results (Confidence Level: 95%)
- Null hypothesis can NOT be rejected. (A >= B)
p-value = 0.5018
- Purchase CVR (A): 1.28%
- Purchase CVR (B): 1.28%
